In [1]:
import imageio, json, os, torch, einops, math, tqdm
import numpy as np
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt
import time

from src.datasets.Dataset_CholecSeg_preprocessed import Dataset_CholecSeg_preprocessed
from src.utils.BaselineConfigs import EXP_OctreeNCA3D

torch.set_grad_enabled(False)

/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nlemke/remote/miniconda3/envs/nca3/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
from src.models.Model_OctreeNCA_3d_patching2 import OctreeNCA3DPatch2


model_path = "/local/scratch/clmn1/octree_study_new/Experiments/cholecfFixAbl_none_10_1.0_16_1_1.0_0.99_OctreeNCASegmentation/"
with open(os.path.join(model_path, "config.json")) as f:
    config = json.load(f) 

exp = EXP_OctreeNCA3D().createExperiment(config, detail_config={}, 
                                        dataset_class=Dataset_CholecSeg_preprocessed, dataset_args = {
                                        })

model: OctreeNCA3DPatch2 = exp.model
assert isinstance(model, OctreeNCA3DPatch2)
model.eval()

Loaded configuration is missing keys: '['experiment.run_hash']'. Check if you are loading the correct experiment.
Timeout Error:  The file lock '/local/scratch/clmn1/octree_study_new/Aim/.aim/locks/a389bc1d54004f328d017483.softlock' could not be acquired.


Force-releasing locks for Run 'a389bc1d54004f328d017483'. Data corruption may occur if there is active process writing to Run 'a389bc1d54004f328d017483'.


Reload State 2000

-------- Experiment Setup --------
{
    "experiment.name": "cholecfFixAbl_none_10_1.0_16_1_1.0_0.99",
    "experiment.description": "OctreeNCASegmentation",
    "model.output_channels": 5,
    "model.channel_n": 16,
    "model.fire_rate": 0.5,
    "model.kernel_size": [
        3,
        3,
        3,
        3,
        3
    ],
    "model.hidden_size": 64,
    "model.batchnorm_track_running_stats": false,
    "model.train.patch_sizes": [
        [
            60,
            106,
            20
        ],
        [
            60,
            106,
            20
        ],
        null,
        null,
        null
    ],
    "model.train.loss_weighted_patching": false,
    "model.eval.patch_wise": false,
    "model.octree.res_and_steps": [
        [
            [
                240,
                432,
                80
            ],
            10
        ],
        [
            [
                120,
                216,
                40
            ],
   

OctreeNCA3DPatch2(
  (backbone_ncas): ModuleList(
    (0-4): 5 x BasicNCA3DFast(
      (fc0): Conv3d(32, 64, kernel_size=(1, 1, 1), stride=(1, 1, 1))
      (fc1): Conv3d(64, 13, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (conv): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), groups=16, padding_mode=reflect)
      (bn): Identity()
    )
  )
)

In [3]:
def downscale(x: torch.Tensor, out_size):
    x = model.align_tensor_to(x, "BCHWD")
    model.remove_names(x)

    out = F.interpolate(x, size=out_size)
    out.names = ('B', 'C', 'H', 'W', 'D')
    x.names = ('B', 'C', 'H', 'W', 'D')
    return out

In [33]:
video_path = "/local/scratch/Cholec80/cholec80_full_set/videos/video01.mp4"
video_reader = imageio.get_reader(video_path)
n_frames = video_reader.get_meta_data()['duration'] * video_reader.get_meta_data()['fps']

num_seconds = 6

video = []
for frame in range(int(video_reader.get_meta_data()['fps'] * num_seconds)):
    image = video_reader.get_data(frame)
    video.append(image[None, ...])

video = np.concatenate(video, axis=0)
video = einops.rearrange(video, 't h w c ->  h w (t c)')


outstacks = []
for i in range(math.ceil(video.shape[-1] / 500)):
    outstack = cv2.resize(video[..., i*500:(i+1)*500], (424, 240))
    outstacks.append(outstack)
video = np.concatenate(outstacks, axis=-1)
video = einops.rearrange(video, 'h w (t c) -> t h w c', c=3).astype(np.float32)
video /= 255.0
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
video -= mean
video /= std
video.shape

(150, 240, 424, 3)

In [34]:
video_tensor = torch.from_numpy(einops.rearrange(video, 'D H W C -> 1 H W D C'))
video_tensor.names = ('B', 'H', 'W', 'D', 'C')
computed_resolutions = model.compute_octree_res(video_tensor)
print(computed_resolutions)

seed = torch.zeros(1, *computed_resolutions[-1], model.channel_n,
                                dtype=torch.float, device=model.device, 
                                names=('B', 'H', 'W', 'D', 'C'))
temp = downscale(video_tensor, computed_resolutions[-1])
temp = model.align_tensor_to(temp, "BHWDC")
model.remove_names(temp)
model.remove_names(seed)
seed[:,:,:,:,:model.input_channels] = temp

[[240, 424, 150], [120, 212, 75], [60, 106, 38], [30, 53, 19], [15, 27, 10]]


In [35]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
start = time.time()
state = model.backbone_ncas[4](seed, steps=model.inference_steps[4], fire_rate=model.fire_rate)

state = einops.rearrange(state, '1 H W D C -> 1 C H W D')
state = torch.nn.Upsample(size=computed_resolutions[3], mode='nearest')(state)
temp = F.interpolate(einops.rearrange(video_tensor, "1 h w t c -> 1 c h w t"), size=computed_resolutions[3])
state[0,:model.input_channels,:,:,:] = temp[0]
state = einops.rearrange(state, '1 C H W D -> 1 H W D C')
state = model.backbone_ncas[3](state, steps=model.inference_steps[3], fire_rate=model.fire_rate)

state = einops.rearrange(state, '1 H W D C -> 1 C H W D')
state = torch.nn.Upsample(size=computed_resolutions[2], mode='nearest')(state)
temp = F.interpolate(einops.rearrange(video_tensor, "1 h w t c -> 1 c h w t"), size=computed_resolutions[2])
state[0,:model.input_channels,:,:,:] = temp[0]
state = einops.rearrange(state, '1 C H W D -> 1 H W D C')
state = model.backbone_ncas[2](state, steps=model.inference_steps[2], fire_rate=model.fire_rate)

state = einops.rearrange(state, '1 H W D C -> 1 C H W D')
state = torch.nn.Upsample(size=computed_resolutions[1], mode='nearest')(state)
temp = F.interpolate(einops.rearrange(video_tensor, "1 h w t c -> 1 c h w t"), size=computed_resolutions[1])
state[0,:model.input_channels,:,:,:] = temp[0]
state = einops.rearrange(state, '1 C H W D -> 1 H W D C')
state = model.backbone_ncas[1](state, steps=model.inference_steps[1], fire_rate=model.fire_rate)

state = einops.rearrange(state, '1 H W D C -> 1 C H W D')
state = torch.nn.Upsample(size=computed_resolutions[0], mode='nearest')(state)
temp = F.interpolate(einops.rearrange(video_tensor, "1 h w t c -> 1 c h w t"), size=computed_resolutions[0])
state[0,:model.input_channels,:,:,:] = temp[0]
state = einops.rearrange(state, '1 C H W D -> 1 H W D C')
state = model.backbone_ncas[0](state, steps=model.inference_steps[0], fire_rate=model.fire_rate)

end = time.time()
print("Time taken: ", end-start)
print(torch.cuda.max_memory_allocated() / 1000**2)

OutOfMemoryError: CUDA out of memory. Tried to allocate 958.00 MiB. GPU 0 has a total capacity of 23.64 GiB of which 830.69 MiB is free. Including non-PyTorch memory, this process has 22.81 GiB memory in use. Of the allocated memory 20.72 GiB is allocated by PyTorch, and 1.66 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)